For Reference, see ESRI's <A href="https://developers.arcgis.com/python/guide/introduction-to-the-spatially-enabled-dataframe/">Introduction to the Spatially Enabled DataFrame</a>

In [1]:
import arcpy
from arcgis.gis import GIS

import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

import numpy as np

In [2]:
# We'll load this object for mapping

gis = GIS()

In [3]:
# Define your working directory

arcpy.env.workspace = "data/sedf_example.gdb"

In [4]:
# List the layers in your working directory

fclist = arcpy.ListFeatureClasses()
print(fclist)

['storm_points', 'watersheds']


In [5]:
sdf = pd.DataFrame.spatial.from_featureclass("storm_points")
watersheds = pd.DataFrame.spatial.from_featureclass("watersheds")

In [6]:
watersheds.head(3)

,OBJECTID,Watershed,TMDL_Water,Phase,Area_Ac,TIA_Ac,TIA_Pct,GlobalID,SHAPE
0,1,Roanoke River above Masons Creek,Roanoke River 2,,40530.632918,0.0,0.0,a6b2fd42-5f46-4486-9101-77d3554abd41,"{'rings': [[[11024671.733272344, 3652589.89073..."
1,2,Back Creek,Back Creek,,37561.660519,0.0,0.0,abbb57b8-7d38-4ed4-b70d-d41ed83ea617,"{'rings': [[[11086384.49403292, 3616511.612173..."
2,3,Barnhardt Creek,"Mud Lick Creek, Murray Run, and Ore Branch",,2630.613126,0.0,0.0,4e65165c-536f-4482-b9a1-ba88363f735a,"{'rings': [[[11041922.518980667, 3625650.97709..."


In [7]:
sdf.head(10)

,OBJECTID,UNIQUEID,WATERSHED,SHAPE
0,1,1001,Back Creek,"{'x': 11088195.428403169, 'y': 3611249.6433663..."
1,2,1002,Back Creek,"{'x': 11025054.510393918, 'y': 3597777.9813333..."
2,3,1003,Carvin Creek,"{'x': 11058512.062245086, 'y': 3680230.5989080..."
3,4,1004,Roanoke River above Masons Creek,"{'x': 11012522.582701921, 'y': 3639462.3290668..."
4,5,1005,Roanoke River above Masons Creek,"{'x': 11018717.112635672, 'y': 3637251.9217422..."
5,6,0,,"{'x': 11064369.069231838, 'y': 3643759.9864820..."
6,7,0,,"{'x': 11031997.086731836, 'y': 3641956.6734875..."
7,8,0,,"{'x': 11039318.002208084, 'y': 3643422.6692432..."
8,9,0,,"{'x': 11054564.431032762, 'y': 3675578.6308498..."
9,10,0,,"{'x': 11051263.145640597, 'y': 3586817.0288458..."


# Visualize Data

In [8]:
m = gis.map("Roanoke, VA", 10)
watersheds.spatial.plot(kind='map',map_widget=m,alpha=.5,renderer_type='u',col='Watershed',pallette='jet')
sdf.spatial.plot(kind='map',map_widget=m,renderer_type='u',col='WATERSHED',pallette='jet')
m

MapView(layout=Layout(height='400px', width='100%'), zoom=10.0)

# Add IDs

In [9]:
max_id = np.max(sdf['UNIQUEID'])

idx = sdf['UNIQUEID'] == 0
n = np.sum(idx)
new_ids = np.arange(max_id+1,max_id+n+1)

print(new_ids)

[1006 1007 1008 1009 1010]


In [10]:
sdf.loc[idx,'UNIQUEID'] = new_ids
sdf.head(10)

,OBJECTID,UNIQUEID,WATERSHED,SHAPE
0,1,1001,Back Creek,"{'x': 11088195.428403169, 'y': 3611249.6433663..."
1,2,1002,Back Creek,"{'x': 11025054.510393918, 'y': 3597777.9813333..."
2,3,1003,Carvin Creek,"{'x': 11058512.062245086, 'y': 3680230.5989080..."
3,4,1004,Roanoke River above Masons Creek,"{'x': 11012522.582701921, 'y': 3639462.3290668..."
4,5,1005,Roanoke River above Masons Creek,"{'x': 11018717.112635672, 'y': 3637251.9217422..."
5,6,1006,,"{'x': 11064369.069231838, 'y': 3643759.9864820..."
6,7,1007,,"{'x': 11031997.086731836, 'y': 3641956.6734875..."
7,8,1008,,"{'x': 11039318.002208084, 'y': 3643422.6692432..."
8,9,1009,,"{'x': 11054564.431032762, 'y': 3675578.6308498..."
9,10,1010,,"{'x': 11051263.145640597, 'y': 3586817.0288458..."


# Add Watersheds

In [11]:
joined_df = sdf.spatial.join(watersheds.loc[:,['Watershed','SHAPE']])
joined_df.head()

# Note carefully that this output has been scrambled (rows are not in the same order)!  
# Indices (leftmost-column) no longer map as they originally did.
# This really should not happen!

,OBJECTID,UNIQUEID,WATERSHED,SHAPE,index_right,Watershed
0,1,1001,Back Creek,"{'x': 11088195.428403169, 'y': 3611249.6433663...",1,Back Creek
1,2,1002,Back Creek,"{'x': 11025054.510393918, 'y': 3597777.9813333...",1,Back Creek
2,10,1010,,"{'x': 11051263.145640597, 'y': 3586817.0288458...",1,Back Creek
3,3,1003,Carvin Creek,"{'x': 11058512.062245086, 'y': 3680230.5989080...",3,Carvin Creek
4,6,1006,,"{'x': 11064369.069231838, 'y': 3643759.9864820...",3,Carvin Creek


In [12]:
# How you locate null values depends on your dataset.  In this case, they are interpreted as space
# (NOT an empty string!)
idx = joined_df['WATERSHED'] == ' '
joined_df.loc[idx,'WATERSHED'] = joined_df.loc[idx,'Watershed']
joined_df.head()

,OBJECTID,UNIQUEID,WATERSHED,SHAPE,index_right,Watershed
0,1,1001,Back Creek,"{'x': 11088195.428403169, 'y': 3611249.6433663...",1,Back Creek
1,2,1002,Back Creek,"{'x': 11025054.510393918, 'y': 3597777.9813333...",1,Back Creek
2,10,1010,Back Creek,"{'x': 11051263.145640597, 'y': 3586817.0288458...",1,Back Creek
3,3,1003,Carvin Creek,"{'x': 11058512.062245086, 'y': 3680230.5989080...",3,Carvin Creek
4,6,1006,Carvin Creek,"{'x': 11064369.069231838, 'y': 3643759.9864820...",3,Carvin Creek


In [13]:
joined_df = joined_df.drop(labels=['index_right','Watershed'],axis=1)

In [14]:
joined_df = joined_df.sort_values(by='UNIQUEID')
joined_df.head(10)

,OBJECTID,UNIQUEID,WATERSHED,SHAPE
0,1,1001,Back Creek,"{'x': 11088195.428403169, 'y': 3611249.6433663..."
1,2,1002,Back Creek,"{'x': 11025054.510393918, 'y': 3597777.9813333..."
3,3,1003,Carvin Creek,"{'x': 11058512.062245086, 'y': 3680230.5989080..."
6,4,1004,Roanoke River above Masons Creek,"{'x': 11012522.582701921, 'y': 3639462.3290668..."
7,5,1005,Roanoke River above Masons Creek,"{'x': 11018717.112635672, 'y': 3637251.9217422..."
4,6,1006,Carvin Creek,"{'x': 11064369.069231838, 'y': 3643759.9864820..."
8,7,1007,Mason Creek,"{'x': 11031997.086731836, 'y': 3641956.6734875..."
9,8,1008,Peters Creek,"{'x': 11039318.002208084, 'y': 3643422.6692432..."
5,9,1009,Carvin Creek,"{'x': 11054564.431032762, 'y': 3675578.6308498..."
2,10,1010,Back Creek,"{'x': 11051263.145640597, 'y': 3586817.0288458..."


# Write out the data

In [15]:
# We might really just overwrite the data, but for clarity in pedagogy we'll write out a separate layer

joined_df.spatial.to_featureclass('storm_points_updated')

'D:\\data\\github\\analysis_in_gis\\data\\sedf_example.gdb\\storm_points_updated'

# Could we see that process all at once, with no frills?

In [17]:
import pandas as pd
import numpy as np

import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor

# Define workspace
arcpy.env.workspace = "data/sedf_example.gdb"

# Load layers
sdf = pd.DataFrame.spatial.from_featureclass("storm_points")
watersheds = pd.DataFrame.spatial.from_featureclass("watersheds")

# Add new IDs
max_id = np.max(sdf['UNIQUEID'])
idx = sdf['UNIQUEID'] == 0
n = np.sum(idx)
new_ids = np.arange(max_id+1,max_id+n+1)
sdf.loc[idx,'UNIQUEID'] = new_ids

# Add missing watersheds
joined_df = sdf.spatial.join(watersheds.loc[:,['Watershed','SHAPE']])
idx = joined_df['WATERSHED'] == ' '
joined_df.loc[idx,'WATERSHED'] = joined_df.loc[idx,'Watershed']
joined_df = joined_df.drop(labels=['index_right','Watershed'],axis=1)

# Optionally, sort
joined_df = joined_df.sort_values(by='UNIQUEID')

# Write out the data
joined_df.spatial.to_featureclass('storm_points_updated')

'D:\\data\\github\\analysis_in_gis\\data\\sedf_example.gdb\\storm_points_updated'